In [3]:
import requests
import json
from IPython.display import display, Image, HTML

# 2 lists so user input can be transformed into workable prefix
langs = ["en","it","nl","es","de"]
langsfull = ["English","Italian","Dutch","Spanish","German"]

print("Welcome! With this program you can quickly get information from Wikipedia!\n")
print("Choose one of the following langauges:\n")
for l in langsfull:
    print(f"- {l}") # showing available languages

#asking for input
inputprefix = input("Language: \n").lower().capitalize().strip()
while inputprefix not in langsfull: #securing the right input
    print("Please choose a langauge out of the given options.")
    inputprefix = input("Language: \n").lower().capitalize().strip()

print("Got that! Now choose your article by name!\n")
inputdata = input("Article name: \n").lower().title().strip()
print(f"\nYou chose {inputprefix} as language and {inputdata} as article name.") 
inputdata = inputdata.replace(" ","_") #to transform UserInput into computer readable parts for API calls

#locating and replacing
location = langsfull.index(inputprefix) 
prefix = langs[location] 
inputurl = (f"https://{prefix}.wikipedia.org/api/rest_v1/page/summary/{inputdata}")
req = requests.get(inputurl)
statuscode = req.status_code

# statuscode function so I can run it multiple times
def check_statuscode():
    if statuscode is 200:
       print(f"\nStatuscode {statuscode} - Which means the query succesfull! We can get the data! (see below...) \n")
    else:
       print(f"\nOops.. error error... iets gaat er fout. Error {statuscode}!")
       %exit #quit() of exit() werken niet in ipython notebook...
check_statuscode()

data = json.loads(req.text)
calls = ["description","coordinates","thumbnail","extract_html"]

# returning all wanted info to the user
display(HTML(f'<b><u>Information:</b></u><br>'))
print(f"\nTitle: {data['title']}") 
for call in calls:
    if call not in data: # general error if something goes wrong
        print(f"{call.capitalize()}: No {call} exists.")
    else:
        if call == "description":
            print(f"Description: {data['description']}")
        if call == "thumbnail":
            thumbsrc = f"{data['thumbnail']['source']}"
            img = Image(url = thumbsrc)
            display(img)
        if call == "extract_html":
            extract = HTML(data['extract_html'])
            display(extract)
        if call == "coordinates":
            lat = data['coordinates']['lat']
            lon = data['coordinates']['lon']
            maps_url = (f"http://maps.google.com/?q=loc:{lat}+{lon}")
            display(HTML(f'Location: <a href="{maps_url}">Visit this place on Google Maps!</a><br><br>'))

# swapping from summary to related articles input
req = requests.get(inputurl.replace("summary","related"))
display(HTML(f'<b><u>Related Results:</b></u><br>'))
check_statuscode()
data = json.loads(req.text)
related_results = []
looplist = ["title","description","url"]
index = 0

# writing the values from the json file to the dictionaries in the lists 
for item in looplist:
    title = data["pages"][index]["displaytitle"]
    if "description" in data["pages"][index]: 
        description = data["pages"][index]["description"]
    else: 
        description == "No description available."
    contenturls = data["pages"][index]["content_urls"]["desktop"]["page"]
    dict_item = {
        "Title": title,
        "Description": description,
        "URL": contenturls}
    add = related_results.append(dict_item)
    index = index + 1

# related results output from dictionaries    
for line in related_results:
        print(f'Title: {line["Title"]}')
        print(f'Description: {line["Description"]}')
        print(f'Desktop URL: {line["URL"]}')
        display(HTML(f'<a href="{line["URL"]}">Visit the Wiki page.</a><br><br>'))

Welcome! With this program you can quickly get information from Wikipedia!

Choose one of the following langauges:

- English
- Italian
- Dutch
- Spanish
- German
Got that! Now choose your article by name!


You chose Dutch as language and Sdfsdf as article name.

Oops.. error error... iets gaat er fout. Error 404!


UsageError: Line magic function `%exit` not found.
